Prediction of credit default

This is an end to end machine learning project to predict credit default
The note book will consider the following
1. Data cleaning
2. Data exploration EDA
3. Prediction and model evaluation
4. Model optimization

In [29]:
# import libraries
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
# import pydot
%matplotlib inline
plt.style.use('seaborn-bright')

import datetime as dt
import timeit
#machine learning

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix


# Dataset is extremely large and current workspace doesn't allow to read all of it

There are several columns that are not useful (at least for now) to predict defaulters. on the other hand there are columns to leak information about loans. 

irrelevant_or_leak_info = ["id", "member_id", "funded_amnt", "funded_amnt_inv", "grade", "sub_grade", "emp_title", "issue_d","zip_code", "out_prncp", "out_prncp_inv", "total_pymnt", "total_pymnt_inv", "total_rec_prncp",
          "total_rec_int", "total_rec_late_fee", "recoveries", "collection_recovery_fee", "last_pymnt_d", "last_pymnt_amnt"]
          
  or more specifically the columns need in order to load a large dataset into this notebook version due to computing power is the cols_to_use
          
Additionally might be easier to read in chunks

In [ ]:
#Columns to read

In [2]:
cols_to_use =['loan_amnt',
 'term',
 'int_rate',
 'installment',
 'emp_length',
 'home_ownership',
 'annual_inc',
 'verification_status',
 'loan_status',
 'pymnt_plan',
 'purpose',
 'title',
 'addr_state',
 'dti',
 'delinq_2yrs',
 'earliest_cr_line',
 'inq_last_6mths',
 'open_acc',
 'pub_rec',
 'revol_bal',
 'revol_util',
 'total_acc',
 'initial_list_status',
 'last_credit_pull_d',
 'collections_12_mths_ex_med',
 'policy_code',
 'application_type',
 'acc_now_delinq',
 'chargeoff_within_12_mths',
 'delinq_amnt',
 'pub_rec_bankruptcies',
 'tax_liens']

# len(["id", "member_id", "funded_amnt", "funded_amnt_inv", "grade", "sub_grade", "emp_title", "issue_d","zip_code", "out_prncp", "out_prncp_inv", "total_pymnt", "total_pymnt_inv", "total_rec_prncp",
#          "total_rec_int", "total_rec_late_fee", "recoveries", "collection_recovery_fee", "last_pymnt_d", "last_pymnt_amnt"])


In [3]:
credit_data = []
for chunk in pd.read_csv('Modified_lending_data.csv', chunksize=25000, index_col=0):
    credit_data.append(chunk)

    ### To only read a set number of 
# credit_data.read_csv('Modified_lending_data.csv',  nrows =350000)
credit_data=pd.concat(credit_data)
print('Rows: ', credit_data.shape[0], ' Number of columns: ', credit_data.shape[1], '\n')
credit_data.head(10)

Rows:  816935  Number of columns:  52 



,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,last_pymnt_amnt,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens
0,7255326,NaN,25000.0,25000.0,25000.0,36 months,12.99,842.23,B,B4,...,11027.09,Feb-2019,0.0,1.0,Individual,0.0,0.0,0.0,0.0,0.0
1,70865715,NaN,8000.0,8000.0,8000.0,36 months,11.47,263.70,B,B5,...,263.45,Mar-2019,0.0,1.0,Individual,0.0,0.0,0.0,0.0,0.0
2,41079864,NaN,16000.0,16000.0,16000.0,36 months,6.39,489.59,A,A2,...,650.64,Mar-2017,0.0,1.0,Individual,0.0,0.0,0.0,0.0,0.0
3,69902853,NaN,15000.0,15000.0,15000.0,36 months,13.99,512.60,C,C4,...,512.60,Jan-2018,0.0,1.0,Individual,0.0,0.0,0.0,0.0,0.0
4,51005594,NaN,25000.0,25000.0,25000.0,36 months,7.89,782.15,A,A5,...,782.15,Mar-2019,0.0,1.0,Individual,0.0,0.0,0.0,0.0,0.0
5,54998888,NaN,15000.0,15000.0,15000.0,36 months,13.99,512.60,C,C4,...,515.76,Mar-2019,0.0,1.0,Individual,0.0,0.0,0.0,0.0,0.0
6,38447561,NaN,6000.0,6000.0,6000.0,36 months,18.54,218.55,E,E1,...,18.32,Apr-2015,0.0,1.0,Individual,0.0,0.0,0.0,0.0,0.0
7,114508765,NaN,20000.0,20000.0,20000.0,36 months,7.07,618.19,A,A2,...,17621.42,Mar-2018,0.0,1.0,Individual,0.0,0.0,0.0,0.0,0.0
8,135963462,NaN,10000.0,10000.0,10000.0,36 months,16.46,353.85,C,C5,...,9917.51,Jan-2019,0.0,1.0,Individual,0.0,0.0,0.0,1.0,0.0
9,55272020,NaN,10000.0,10000.0,10000.0,36 months,14.65,344.95,C,C5,...,1.98,Jan-2019,0.0,1.0,Individual,0.0,0.0,0.0,0.0,0.0


In [9]:
credit_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 816935 entries, 0 to 816934
Data columns (total 52 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   id                          816935 non-null  int64  
 1   member_id                   0 non-null       float64
 2   loan_amnt                   816935 non-null  float64
 3   funded_amnt                 816935 non-null  float64
 4   funded_amnt_inv             816935 non-null  float64
 5   term                        816935 non-null  object 
 6   int_rate                    816935 non-null  float64
 7   installment                 816935 non-null  float64
 8   grade                       816935 non-null  object 
 9   sub_grade                   816935 non-null  object 
 10  emp_title                   764696 non-null  object 
 11  emp_length                  769102 non-null  object 
 12  home_ownership              816935 non-null  object 
 13  annual_inc    

In [10]:
## Remove columns that are not useful or leak infomation about loan_status

In [4]:
credit_data = credit_data[cols_to_use]
credit_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 816935 entries, 0 to 816934
Data columns (total 32 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   loan_amnt                   816935 non-null  float64
 1   term                        816935 non-null  object 
 2   int_rate                    816935 non-null  float64
 3   installment                 816935 non-null  float64
 4   emp_length                  769102 non-null  object 
 5   home_ownership              816935 non-null  object 
 6   annual_inc                  816935 non-null  float64
 7   verification_status         816935 non-null  object 
 8   loan_status                 816935 non-null  object 
 9   pymnt_plan                  816935 non-null  object 
 10  purpose                     816935 non-null  object 
 11  title                       806917 non-null  object 
 12  addr_state                  816935 non-null  object 
 13  dti           

## Evaluating target variable

In [5]:
target = 'loan_status'

In [11]:
credit_data.loan_status.value_counts(normalize = True)

Fully Paid     0.800202
Charged Off    0.199798
Name: loan_status, dtype: float64

In [6]:
# change labels 1 for fully paid and 0 for Charged off

status_mapping= {
    'loan_status':{
        'Fully Paid':1,
        'Charged Off' :0
    }
}

In [7]:
credit_data = credit_data.replace(status_mapping)
credit_data.loan_status.value_counts(normalize = True)

1    0.800202
0    0.199798
Name: loan_status, dtype: float64

In [9]:
credit_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 816935 entries, 0 to 816934
Data columns (total 32 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   loan_amnt                   816935 non-null  float64
 1   term                        816935 non-null  object 
 2   int_rate                    816935 non-null  float64
 3   installment                 816935 non-null  float64
 4   emp_length                  769102 non-null  object 
 5   home_ownership              816935 non-null  object 
 6   annual_inc                  816935 non-null  float64
 7   verification_status         816935 non-null  object 
 8   loan_status                 816935 non-null  object 
 9   pymnt_plan                  816935 non-null  object 
 10  purpose                     816935 non-null  object 
 11  title                       806917 non-null  object 
 12  addr_state                  816935 non-null  object 
 13  dti           

In [11]:
#look for columns that have low variablility

[[col, len(pd.unique(credit_data[col]))] for col in credit_data.columns]
# credit_data.nunique(axis=1)

[['loan_amnt', 1532],
 ['term', 2],
 ['int_rate', 640],
 ['installment', 72779],
 ['emp_length', 12],
 ['home_ownership', 6],
 ['annual_inc', 45673],
 ['verification_status', 3],
 ['loan_status', 2],
 ['pymnt_plan', 1],
 ['purpose', 14],
 ['title', 40269],
 ['addr_state', 51],
 ['dti', 6299],
 ['delinq_2yrs', 27],
 ['earliest_cr_line', 727],
 ['inq_last_6mths', 9],
 ['open_acc', 78],
 ['pub_rec', 33],
 ['revol_bal', 71492],
 ['revol_util', 1294],
 ['total_acc', 134],
 ['initial_list_status', 2],
 ['last_credit_pull_d', 138],
 ['collections_12_mths_ex_med', 14],
 ['policy_code', 1],
 ['application_type', 3],
 ['acc_now_delinq', 9],
 ['chargeoff_within_12_mths', 12],
 ['delinq_amnt', 1417],
 ['pub_rec_bankruptcies', 12],
 ['tax_liens', 32]]

In [8]:
# drop cols with just one variable
for column in credit_data:
    if len(pd.unique(credit_data[column])) <=1:
        credit_data.drop(column, axis = 1, inplace =True)
        print(column, ': has been removed')
credit_data.shape

pymnt_plan : has been removed
policy_code : has been removed


(816935, 30)

##### Drop all nan's as we have a big dataset. We can come and take a look 

In [9]:
credit_data.dropna(inplace=True)
# see if proportions of loan_status has changed
credit_data.loan_status.value_counts(normalize=True)

1    0.805438
0    0.194562
Name: loan_status, dtype: float64

## Process object columns, modify and 'dummify' where neccessary!

In [11]:
object_cols = credit_data.select_dtypes(include=['object']).columns.to_list()
credit_data[object_cols]

,term,emp_length,home_ownership,verification_status,purpose,title,addr_state,earliest_cr_line,initial_list_status,last_credit_pull_d,application_type
0,36 months,10+ years,MORTGAGE,Verified,debt_consolidation,Debt consolidation,CO,Feb-1990,f,Feb-2019,Individual
1,36 months,10+ years,RENT,Verified,other,Other,CA,Jul-2002,w,Mar-2019,Individual
2,36 months,< 1 year,RENT,Not Verified,credit_card,Credit card refinancing,IL,Mar-1987,f,Mar-2017,Individual
3,36 months,7 years,MORTGAGE,Verified,debt_consolidation,Debt consolidation,TX,Mar-1990,w,Jan-2018,Individual
4,36 months,4 years,MORTGAGE,Source Verified,credit_card,Credit card refinancing,MI,Jun-2002,w,Mar-2019,Individual
...,...,...,...,...,...,...,...,...,...,...,...
816928,60 months,2 years,MORTGAGE,Source Verified,debt_consolidation,Debt consolidation,TX,Jan-1985,w,Nov-2016,Individual
816929,60 months,4 years,RENT,Source Verified,credit_card,Credit card refinancing,MO,Nov-2004,w,Mar-2019,Individual
816930,36 months,5 years,RENT,Source Verified,debt_consolidation,Debt consolidation,CT,Jul-2005,f,Mar-2019,Individual
816931,36 months,< 1 year,MORTGAGE,Not Verified,debt_consolidation,Debt consolidation,WI,Oct-1997,w,Jan-2019,Individual


In [10]:
 ### These two columns are dates which might require processing to get signal so drop ..earliest_cr_line, last_credit_pull_d

In [14]:
# credit_data.drop(['earliest_cr_line', 'last_credit_pull_d'], axis = 1, inplace = True)
# object_cols = credit_data.select_dtypes(include=['object']).columns.to_list()
# credit_data[object_cols]

In [14]:
#Number of unique categories
credit_data[object_cols].nunique()

term                       2
emp_length                11
home_ownership             6
verification_status        3
purpose                   14
title                  38527
addr_state                51
initial_list_status        2
application_type           2
dtype: int64

##### The title and purpose column contain the same information but title has ~ 40K description therefore it will be dropped. 
**addr_state** column has 51 uniques which will result in 51 new columns. Lets remove for now

In [15]:
credit_data.drop(['title', 'addr_state'], axis = 1, inplace = True)
object_cols = credit_data.select_dtypes(include=['object']).columns.to_list()
credit_data[object_cols]

,term,emp_length,home_ownership,verification_status,purpose,initial_list_status,application_type
0,36 months,10+ years,MORTGAGE,Verified,debt_consolidation,f,Individual
1,36 months,10+ years,RENT,Verified,other,w,Individual
2,36 months,< 1 year,RENT,Not Verified,credit_card,f,Individual
3,36 months,7 years,MORTGAGE,Verified,debt_consolidation,w,Individual
4,36 months,4 years,MORTGAGE,Source Verified,credit_card,w,Individual
...,...,...,...,...,...,...,...
816928,60 months,2 years,MORTGAGE,Source Verified,debt_consolidation,w,Individual
816929,60 months,4 years,RENT,Source Verified,credit_card,w,Individual
816930,36 months,5 years,RENT,Source Verified,debt_consolidation,f,Individual
816931,36 months,< 1 year,MORTGAGE,Not Verified,debt_consolidation,w,Individual


Emp temp column is a string but could be converted to numbers and then integer because it is ordinal. 10 years of working is higher than
2.
Clean the column and covert to float

In [16]:
# replace < 1 year with 0.5 i.e. six months
mapping_dict = {
    "emp_length": {
        "10+ years": 10,
        "9 years": 9,
        "8 years": 8,
        "7 years": 7,
        "6 years": 6,
        "5 years": 5,
        "4 years": 4,
        "3 years": 3,
        "2 years": 2,
        "1 year": 1,
        "< 1 year": 0.5,
        "n/a": 0
    }
}

In [17]:
credit_data = credit_data.replace(mapping_dict)

In [18]:
credit_data.emp_length = pd.to_numeric(credit_data.emp_length)
object_cols = credit_data.select_dtypes(include=['object']).columns.to_list()
print(credit_data.emp_length.value_counts())
credit_data[object_cols]

10.0    264855
2.0      73239
0.5      64816
3.0      64416
1.0      52722
5.0      50727
4.0      48237
6.0      37524
8.0      36477
7.0      35816
9.0      30670
Name: emp_length, dtype: int64


,term,home_ownership,verification_status,purpose,initial_list_status,application_type
0,36 months,MORTGAGE,Verified,debt_consolidation,f,Individual
1,36 months,RENT,Verified,other,w,Individual
2,36 months,RENT,Not Verified,credit_card,f,Individual
3,36 months,MORTGAGE,Verified,debt_consolidation,w,Individual
4,36 months,MORTGAGE,Source Verified,credit_card,w,Individual
...,...,...,...,...,...,...
816928,60 months,MORTGAGE,Source Verified,debt_consolidation,w,Individual
816929,60 months,RENT,Source Verified,credit_card,w,Individual
816930,36 months,RENT,Source Verified,debt_consolidation,f,Individual
816931,36 months,MORTGAGE,Not Verified,debt_consolidation,w,Individual


In [16]:
credit_data.nunique()

loan_amnt                      1526
term                              2
int_rate                        607
installment                   70985
emp_length                       11
home_ownership                    6
annual_inc                    39331
verification_status               3
loan_status                       2
purpose                          14
dti                            5856
delinq_2yrs                      27
inq_last_6mths                    9
open_acc                         77
pub_rec                          33
revol_bal                     70410
revol_util                     1288
total_acc                       133
initial_list_status               2
collections_12_mths_ex_med       13
application_type                  2
acc_now_delinq                    8
chargeoff_within_12_mths         11
delinq_amnt                    1359
pub_rec_bankruptcies             11
tax_liens                        31
dtype: int64

In [21]:
# dummify categorical variables

cols_to_dummify = ['term', 'home_ownership','verification_status', 'purpose', 'initial_list_status','application_type']

In [22]:
credit_data = pd.concat([credit_data,pd.get_dummies(credit_data[cols_to_dummify])], axis = 1)
credit_data.drop(cols_to_dummify,axis = 1, inplace = True)
credit_data.head()

KeyError: "None of [Index(['term', 'home_ownership', 'verification_status', 'purpose',\n       'initial_list_status', 'application_type'],\n      dtype='object')] are in the [columns]"

In [23]:
features = credit_data.columns.tolist()
print(len(features))
features.remove('loan_status')
print('loan_status removed', '\n',len(features))


49
loan_status removed 
 48


### Prediction - logistic regression without balancing

In [36]:
start = timeit.timeit()
now = dt.datetime.now()
lr = LogisticRegression()
predictions=cross_val_predict(lr, credit_data[features], credit_data[target], cv=3)
end = timeit.timeit()
later =dt.datetime.now()
print(end-start)
print('total_time: ',(later - now)/60)

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.030501748202368617
total_time:  0:00:02.263520


In [37]:
# predictions
predictions = pd.Series(predictions)
predictions.value_counts(normalize = True)

1    0.99824
0    0.00176
dtype: float64

In [43]:
tn, fp, fn, tp= confusion_matrix(credit_data[target], predictions).ravel()
 

In [46]:
# predictions = pd.Series(predictions)
# predictions.value_counts(normalize = True)
# tp = len(predictions[(predictions == 1) & credit_data[credit_data.loan_status ==1]])
# fp = len(predictions[(predictions == 1) & credit_data[credit_data.loan_status ==0]])
# fn = len(predictions[(predictions == 0) & credit_data[credit_data.loan_status ==1]])
# tn = len(predictions[(predictions == 0) & credit_data[credit_data.loan_status ==0]])

precision = tp/(tp+fp)
recall = tp/(tp+fp)
print('precision:   ', precision,'\n', 'recall:   ', recall)

precision:    0.8057657862040039 
 recall:    0.8057657862040039


## Balance classes

In [47]:
start = timeit.timeit()
now = dt.datetime.now()
lr = LogisticRegression(class_weight='balanced')
predictions=cross_val_predict(lr, credit_data[features], credit_data[target], cv=3)
end = timeit.timeit()
later =dt.datetime.now()
print(end-start)
print('total_time: ',(later - now)/60)

predictions = pd.Series(predictions)
print(predictions.value_counts(normalize = True))
tn, fp, fn, tp= confusion_matrix(credit_data[target], predictions).ravel()


/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.0009194319136440754
total_time:  0:00:03.763621
1    0.616799
0    0.383201
dtype: float64
precision:    0.8808132212492902 
 recall:    0.8808132212492902


In [48]:
precision = tp/(tp+fp)
recall = tp/(tp+fn)
print('precision:   ', precision,'\n', 'recall:   ', recall)

precision:    0.8808132212492902 
 recall:    0.6745209071337145


In [ ]:
# 5 fold cross val

In [49]:
start = timeit.timeit()
now = dt.datetime.now()
lr = LogisticRegression(class_weight='balanced')
predictions=cross_val_predict(lr, credit_data[features], credit_data[target], cv=3)
end = timeit.timeit()
later =dt.datetime.now()
print(end-start)
print('total_time: ',(later - now),(later - now)/60)

predictions = pd.Series(predictions)
print(predictions.value_counts(normalize = True))
tn, fp, fn, tp= confusion_matrix(credit_data[target], predictions).ravel()
precision = tp/(tp+fp)
recall = tp/(tp+fn)
print('precision:   ', precision,'\n', 'recall:   ', recall)

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.002538834000006318
total_time:  0:03:40.220142 0:00:03.670336
1    0.616799
0    0.383201
dtype: float64
precision:    0.8808132212492902 
 recall:    0.6745209071337145


In [52]:
start = timeit.timeit()
now = dt.datetime.now()
lr = LogisticRegression(class_weight='balanced')
predictions=cross_val_predict(lr, credit_data[features], credit_data[target], cv=3)
end = timeit.timeit()
later =dt.datetime.now()
print(end-start)
print('total_time: ',(later - now),(later - now)/60)

predictions = pd.Series(predictions)
print(predictions.value_counts(normalize = True))
tn, fp, fn, tp= confusion_matrix(credit_data[target], predictions).ravel()
precision = tp/(tp+fp)
recall = tp/(tp+fn)
print('precision:   ', precision,'\n', 'recall:   ', recall)

0.6221560533261149

In [25]:
##. 

start = timeit.timeit()
now = dt.datetime.now()
lr = LogisticRegression(class_weight='balanced')
predictions=cross_val_predict(lr, credit_data[features], credit_data[target], cv=4)
end = timeit.timeit()
later =dt.datetime.now()
print(end-start)
print('total_time: ',(later - now),(later - now)/60)

predictions = pd.Series(predictions)
print(predictions.value_counts(normalize = True))
tn, fp, fn, tp= confusion_matrix(credit_data[target], predictions).ravel()
precision = tp/(tp+fp)
recall = tp/(tp+fn)
print('precision:   ', precision,'\n', 'recall:   ', recall)

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.007561465026810765
total_time:  0:06:53.382195 0:00:06.889703
1    0.606084
0    0.393916
dtype: float64
precision:    0.882008168230796 
 recall:    0.6637023910914801


In [26]:
tpr = tp / (tp + fn)
fpr = fp / (fp + tn)

In [27]:
tpr, fpr

(0.6637023910914801, 0.36755769100629354)

##  Random Forest

In [30]:
start = timeit.timeit()
now = dt.datetime.now()
rf = RandomForestClassifier(class_weight='balanced', random_state=1)
predictions=cross_val_predict(rf, credit_data[features], credit_data[target], cv=3)
end = timeit.timeit()
later =dt.datetime.now()
print(end-start)
print('total_time: ',(later - now),(later - now)/60)

predictions = pd.Series(predictions)
print(predictions.value_counts(normalize = True))
tn, fp, fn, tp= confusion_matrix(credit_data[target], predictions).ravel()
precision = tp/(tp+fp)
recall = tp/(tp+fn)
print('precision:   ', precision,'\n', 'recall:   ', recall)


/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.007018186850473285
total_time:  0:02:29.737144 0:00:02.495619
1    0.919951
0    0.080049
dtype: float64
precision:    0.821686498678979 
 recall:    0.938510353440821


In [31]:
start = timeit.timeit()
now = dt.datetime.now()
rf = RandomForestClassifier(class_weight='balanced', random_state=1)
predictions=cross_val_predict(rf, credit_data[features], credit_data[target], cv=3)
end = timeit.timeit()
later =dt.datetime.now()
print(end-start)
print('total_time: ',(later - now),(later - now)/60)

predictions = pd.Series(predictions)
print(predictions.value_counts(normalize = True))
tn, fp, fn, tp= confusion_matrix(credit_data[target], predictions).ravel()
precision = tp/(tp+fp)
recall = tp/(tp+fn)
print('precision:   ', precision,'\n', 'recall:   ', recall)


/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.008108011214062572
total_time:  0:02:29.537579 0:00:02.492293
1    0.916258
0    0.083742
dtype: float64
precision:    0.822983861117378 
 recall:    0.9362184889060352


In [33]:
tn / (fp + tn)

0.16637341814982742